In [1]:
import sqlalchemy as sa
from sqlalchemy import orm

In [2]:
engine = sa.create_engine('sqlite:///heroes.db')
connection = engine.connect()

In [3]:
class Base(orm.DeclarativeBase):
    pass

In [4]:
class BattleEvents(Base):
    __tablename__ = 'battle_events'

    battle_event_id: orm.Mapped[int] = orm.mapped_column(sa.Integer, primary_key=True)
    battle_event_type_id: orm.Mapped[int] = orm.mapped_column(sa.Integer, nullable=False)
    rubies_gained: orm.Mapped[int] = orm.mapped_column(sa.Integer)
    timestamp: orm.Mapped[int] = orm.mapped_column(sa.Integer, nullable=False)
    battle_participant_id: orm.Mapped[int] = orm.mapped_column(sa.ForeignKey('battle_participants.battle_participant_id'))

    def __repr__(self):
        return f'{type(self).__name__}({self.battle_participant_id}, {self.rubish_gained}, {self.timestamp})'

In [5]:
class BattleParticipants(Base):
    __tablename__ = 'battle_participants'

    battle_participant_id: orm.Mapped[int] = orm.mapped_column(sa.Integer, primary_key=True)
    user_id: orm.Mapped[int] = orm.mapped_column(sa.Integer, nullable=False)
    battle_id: orm.Mapped[int] = orm.mapped_column(sa.Integer, nullable=False)
    hero_id: orm.Mapped[int] = orm.mapped_column(sa.ForeignKey('heroes.hero_id'), nullable=False)

    def __repr__(self):
        return f'{type(self).__name__}({self.battle_participant_id}, {self.user_id}, {self.battle_id})'

In [6]:
class Heroes(Base):
    __tablename__ = 'heroes'

    name: orm.Mapped[str] = orm.mapped_column(sa.String, nullable=False, unique=True)
    hero_id: orm.Mapped[int] = orm.mapped_column(sa.Integer, primary_key=True)

    def __repr__(self):
        return f'{type(self).__name__}({self.name})'

In [10]:
# SEELCT heros.name, Count(be.battle_event_id) as total events
#from heros as h
#JOIN battle_participants as bp ON bp.hero_id = h.hero_id
#JOIN battle_events AS be ON be.battle_participate_id = bp.battle_partticipate_id
#GROUP BY h.name
#ORDER Y TotalEvents DESC;

q = sa.select(
        Heroes.name,
        sa.func.count(BattleEvents.battle_event_id).label('TotalEvents'),
    ) \
    .join(BattleParticipants, BattleParticipants.hero_id == Heroes.hero_id) \
    .join(BattleEvents, BattleEvents.battle_participant_id == BattleParticipants.battle_participant_id) \
    .group_by(Heroes.name) \
    .order_by(sa.desc('TotalEvents'))

dc = connection.execute(q)
print(dc.fetchall())

[('Crareek', 3687), ('Pamble', 2054), ('Huzzt', 2036), ('Tan', 1946), ('Grimm', 1820), ('Jade', 1780), ('Hillstomp', 1736), ('Mako', 1719), ('Grrdy', 1528)]


In [11]:
from sqlalchemy.ext.automap import automap_base

In [13]:
Base = automap_base()
Base.prepare(autoload_with=engine, classname_for_table=lambda cls, table_name, table_obj: table_name.title().replace('_',''))

In [15]:
Battles= Base.classes.Battles
BatleEvents= Base.classes.BattleEvents
Heroes= Base.classes.Heroes
Users= Base.classes.Users
BattleEventTypes= Base.classes.BattleEventTypes
BattleParticipants = Base.classes.BattleParticipants

In [17]:
q = sa.select(
        Heroes.name,
        sa.func.count(BattleEvents.battle_event_id).label('TotalKills'),
    ) \
    .join(BattleParticipants, BattleParticipants.hero_id == Heroes.hero_id) \
    .join(BattleEvents, BattleEvents.battle_participant_id == BattleParticipants.battle_participant_id) \
    .join(BattleEventsTypes, BattleEventsTypes.battle_event_type_id == BattleEvents.battle_event_type_id) \
    .where(BattleEventsTypes.name == "HERO_KILL") 
    .group_by(Heroes.name) \
    .order_by(sa.desc('TotalKills'))

dc = connection.execute(q)
print(dc.fetchall())

NameError: name 'BattleEventsTypes' is not defined